# Classifying Cookies from Non-Cookies
Combine the existing chocolate chip cookie dataset with xxx.

In [ ]:
import os

cookie_images_dir = os.path.join('./dataset/images')

Build a custom cookie/non-cookie dataset. Combines the cookie images with CIFAR-10 - a Keras dataset with 10 categories and 50,000 images.

In [ ]:
import tensorflow as tf
import numpy as np
import pathlib
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_bin_class_dataset(cookie_images_dir, num_non_cookie_samples=1000, target_size=(224, 224)):
    """
    Create a binary classification dataset of cookies vs non-cookies
    
    Parameters:
    cookie_dir: Path to directory containing cookie images
    num_non_cookie_samples: Number of non-cookie images to include
    target_size: Target size for all images
    
    Returns:
    (x_train, y_train), (x_test, y_test): Training and test datasets
    """
    
    # Load and preprocess cookie images
    cookie_images_path = pathlib.Path(cookie_images_dir)
    cookie_images = list(cookie_images_path.glob('*.[jJ][pP][gG]')) + list(cookie_images_path.glob('*.[pP][nN][gG]'))
    
    # Load and preprocess cookie images
    cookies = []
    for img_path in cookie_images:
        img = image.load_img(img_path, target_size=target_size)
        img_array = image.img_to_array(img)
        cookies.append(img_array)
    
    cookies = np.array(cookies)
    
    # Load CIFAR-10 as source of non-cookie images
    (cifar_x_train, cifar_y_train), (cifar_x_test, cifar_y_test) = tf.keras.datasets.cifar10.load_data()
    
    # Resize CIFAR images to match cookie image size
    non_cookies = []
    for img in cifar_x_train[:num_non_cookie_samples]:
        resized = tf.image.resize(img, target_size)
        non_cookies.append(resized)
    
    non_cookies = np.array(non_cookies)
    
    # Combine datasets
    X = np.concatenate([cookies, non_cookies])
    y = np.concatenate([np.ones(len(cookies)), np.zeros(len(non_cookies))])
    
    # Shuffle the dataset
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]
    
    # Split into train and test sets
    split = int(0.8 * len(X))
    x_train, x_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]
    
    # Normalize pixel values
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    
    return (x_train, y_train), (x_test, y_test)

# Data augmentation for training
def create_data_generators(x_train, y_train, x_test, y_test, batch_size=32):
    """
    Create data generators with augmentation for training
    """
    train_datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    
    test_datagen = ImageDataGenerator()
    
    train_generator = train_datagen.flow(
        x_train, y_train,
        batch_size=batch_size
    )
    
    test_generator = test_datagen.flow(
        x_test, y_test,
        batch_size=batch_size
    )
    
    return train_generator, test_generator

Load the dataset and create generators.

In [ ]:
# Load and prepare the dataset
(x_train, y_train), (x_test, y_test) = create_bin_class_dataset(cookie_images_dir)

# Create data generators
train_generator, test_generator = create_data_generators(x_train, y_train, x_test, y_test)

# Print dataset info
print(f"Training samples: {len(x_train)}")
print(f"Test samples: {len(x_test)}")

## Build a model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, Input, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def build_cookie_classifier(input_shape=(224, 224, 3)):
    """
    Build a CNN model for cookie classification with better uncertainty handling
    
    Parameters:
    input_shape: Shape of input images (height, width, channels)
    
    Returns:
    model: Compiled Keras model
    """
    model = Sequential([
        Input(shape=input_shape),
        # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Third Convolutional Block
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Global Average Pooling
        GlobalAveragePooling2D(),
    
        # Dense Layers
        # Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(2, activation='softmax')  # Changed to 2 units with softmax instead of Binary classification
    ])
    
    # Compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']#, tf.keras.metrics.AUC()]
    )
    
    return model

## Train the model

In [ ]:

def train_model(model, train_generator, test_generator, epochs=50):
    """
    Train the model with callbacks for early stopping and learning rate reduction
    
    Parameters:
    model: Compiled Keras model
    train_generator: Training data generator
    test_generator: Validation data generator
    epochs: Maximum number of epochs to train
    
    Returns:
    history: Training history
    """
    # Callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )
    
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-6
    )
    
    # Train the model
    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=test_generator,
        callbacks=[early_stopping, reduce_lr]
    )
    
    return history

def plot_training_history(history):
    """
    Plot training history including accuracy and loss curves
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Accuracy plot
    ax1.plot(history.history['accuracy'], label='Training Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    
    # Loss plot
    ax2.plot(history.history['loss'], label='Training Loss')
    ax2.plot(history.history['val_loss'], label='Validation Loss')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

You can start training for 10 epochs -- this may take a few minutes to run. Increasing the number of epochs reduces the model loss.

| Epochs           | Accuracy    | Loss    |
|------------------|-------------|---------|
| 20               | 0.9999      | 0.0091  |


In [ ]:
# Build and compile the model
model = build_cookie_classifier()

# Print model summary
model.summary()

# Train the model
history = train_model(model, train_generator, test_generator, epochs=12)

# Plot training history
plot_training_history(history)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"\nTest Accuracy: {test_accuracy:.4f}")
print(f"Test AUC: {test_auc:.4f}")

## Predict images

In [ ]:
def predict_image(model, image_path, confidence_threshold=0.8, target_size=(224, 224)):
    """
    Make a prediction on a single image with uncertainty handling
    
    Parameters:
    model: Trained Keras model
    image_path: Path to the image file
    confidence_threshold: Minimum confidence required for a definitive prediction
    target_size: Size to resize the image to
    
    Returns:
    prediction: Class prediction and confidence
    """
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array.astype('float32') / 255.0  # Normalize
    
    # Make prediction
    predictions = model.predict(img_array)[0]
    max_confidence = np.max(predictions)
    predicted_class = np.argmax(predictions)
 
    # Determine prediction based on confidence threshold
    if max_confidence >= confidence_threshold:
        if predicted_class == 1:
            prediction_text = "Cookie"
        else:
            prediction_text = "Not Cookie"
    else:
        prediction_text = "Uncertain"
        
    # Display the image with prediction
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'Prediction: {prediction_text}\nConfidence: {max_confidence:.2%}')
    plt.show()
    
    return {
        'prediction': prediction_text,
        'confidence': max_confidence,
        'probabilities': {
            'not_cookie': predictions[0],
            'cookie': predictions[1]
        }
    }

def print_result(result):
    print("\nPrediction Results:")
    print(f"Class: {result['prediction']}")
    print(f"Confidence: {result['confidence']:.2%}")
    print("\nClass Probabilities:")
    print(f"Not Cookie: {result['probabilities']['not_cookie']:.2%}")
    print(f"Cookie: {result['probabilities']['cookie']:.2%}")

In [ ]:
# Make prediction on new image
result = predict_image(model, "./prediction.jpg")
print_result(result)

In [ ]:
# Make prediction on new image
result = predict_image(model, "./non-cookie 1.jpg")
print_result(result)

In [ ]:
# Make prediction on new image
result = predict_image(model, "./non-cookie 2.png")
print_result(result)

In [ ]:
# Make prediction on new image
result = predict_image(model, "./non-cookie 3.jpg")
print_result(result)

## Export

Export the model to an h5 file.

In [ ]:
model.save('classification-model.h5')